In [1]:
import numpy as np
import mne
from mne.preprocessing import ICA
import sklearn
import matplotlib.pyplot as plt
from mne.preprocessing import annotate_muscle_zscore

In [4]:
# 분석대상 이름 입력
name = input("분석대상 이름을 입력하세요.")
print(f"분석대상 이름은 {name}입니다.")

# Specify the full path to EEG data file (EDF format)
eeg_file_path = rf'H:\Mg_EEG\edf_files\{name}_7200.EDF'

# Load EEG data
eeg_data = mne.io.read_raw_edf(eeg_file_path, preload=True)
print(f"성공적으로 데이터를 불러왔습니다.")

분석대상 이름은 정금례입니다.
Extracting EDF parameters from H:\Mg_EEG\edf_files\정금례_7200.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1440599  =      0.000 ...  7202.995 secs...
성공적으로 데이터를 불러왔습니다.


In [5]:
%matplotlib qt
eeg_data.plot()
plt.show()

Channels marked as bad:
['EOGL-A1', 'EKG1-EKG2']


In [ ]:
eeg_data.info

In [5]:
# Specify the time range of interest
start_time = 0  # Start time in seconds
duration = 7200 # 2 hours in seconds
end_time = start_time + duration # End time in seconds (2 hours later from start_time)
# Mark the time of magnesium infusion
time_of_infusion = 3600  # Time of infusion in seconds

In [6]:
# Crop EEG data
eeg_data_window = eeg_data.copy().crop(tmin=start_time, tmax=7199)

In [ ]:
# Notch filter apply
eeg_data_window.notch_filter(freqs=[60])

In [ ]:
print(eeg_data_window.ch_names)

In [ ]:
eeg_data_trans = eeg_data_window.copy()

eeg_data_trans.drop_channels(eeg_data_window.ch_names[18:])

eeg_data_trans_ch_names = eeg_data_trans.ch_names

ch_names_answer = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
                'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 
                'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 
                'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2', 
                'Fz-Cz', 'Cz-Pz']

# 차이점을 저장할 딕셔너리 초기화
channel_mapping = {}

# 리스트 길이만큼 반복
for i in range(len(eeg_data_trans_ch_names)):
    if eeg_data_trans_ch_names[i] != ch_names_answer[i]:
        # 차이가 있는 경우, 해당 인덱스의 값을 딕셔너리에 저장
        channel_mapping[eeg_data_trans_ch_names[i]] = ch_names_answer[i]

# 결과 출력
print(channel_mapping)

In [ ]:
eeg_data_trans.rename_channels(channel_mapping)

In [ ]:
# Step 1: Identify your current bipolar channels
# This is an example, adjust according to your actual channel names
bipolar_chs = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
                'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 
                'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 
                'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2', 
                'Fz-Cz', 'Cz-Pz']

# Step 2: Create a list of monopolar electrode names
monopolar_chs = ['Fp1', 'F3', 'C3', 'P3', 'O1',
                 'Fp2', 'F4', 'C4', 'P4', 'O2',
                 'F7', 'T3', 'T4', 'T5', 'T6', 'F8',
                 'Fz', 'Cz', 'Pz']

# Step 3: Create a transformation matrix
n_bipolar = len(bipolar_chs)
n_monopolar = len(monopolar_chs)
transform_mat = np.zeros((n_monopolar, n_bipolar))

for i, mono_ch in enumerate(monopolar_chs):
    for j, bi_ch in enumerate(bipolar_chs):
        if mono_ch in bi_ch.split('-'):
            if mono_ch == bi_ch.split('-')[0]:
                transform_mat[i, j] = 1
            else:
                transform_mat[i, j] = -1

# Step 4: Apply the transformation
data = eeg_data_trans.get_data()
monopolar_data = np.dot(transform_mat, data)

# Create a new Raw object with the monopolar data
info = mne.create_info(ch_names=monopolar_chs, sfreq=eeg_data_trans.info['sfreq'], ch_types='eeg')
eeg_data_monopolar = mne.io.RawArray(monopolar_data, info)

# Step 5: Set the new monopolar montage
montage = mne.channels.make_standard_montage('standard_1020')
eeg_data_monopolar.set_montage(montage)

# Now you have a new Raw object with monopolar data in the 10-20 system

In [ ]:
ecg_eog_data_window = eeg_data_window.get_data([18, 19, 20])
#ecg_eog_data_window = eeg_data_window.get_data([18, 19])

new_info = mne.create_info(
    ch_names=['ECG', 'EOG1', 'EOG2'],
    sfreq=eeg_data_window.info['sfreq'],
    ch_types=['ecg', 'eog', 'eog']
)

ecg_eog_raw = mne.io.RawArray(ecg_eog_data_window, new_info)

eeg_data_monopolar.add_channels([ecg_eog_raw], force_update_info=True)

print(eeg_data_monopolar.ch_names)

In [ ]:
eeg_data_monopolar.plot()

In [ ]:
# Detecting artifact by ICA
ica = ICA(n_components=15, random_state=42)
ica.fit(eeg_data_monopolar)

In [ ]:
# ECG detecting
ecg_idx, ecg_scores = ica.find_bads_ecg(eeg_data_monopolar)
# EOG detecting
eog_idx, eog_scores = ica.find_bads_eog(eeg_data_monopolar)
# muscle detecting
muscle_idx, muscle_scores = ica.find_bads_muscle(eeg_data_monopolar)

artifact_idx = list(set(ecg_idx + eog_idx + muscle_idx))
#artifact_idx = list(set(eog_idx + muscle_idx))
#artifact_idx = list(set(muscle_idx))
eeg_data_clean = eeg_data_monopolar.copy()
ica.apply(eeg_data_clean, exclude=artifact_idx)

In [ ]:
# Bandpass filters
eeg_data_clean.filter(l_freq=0.1, h_freq=30)

In [ ]:
eeg_data_clean.plot()

In [ ]:
eeg_data_clean.drop_channels(eeg_data_clean.ch_names[19:])

In [ ]:
eeg_data_clean.save(r'H:\Mg_EEG\edf_files\김득실_7200_clean.fif', overwrite=False)
#eeg_data_clean.save(r'H:\Mg_EEG\edf_files\김득실_7200_clean.fif', overwrite=True) #overwrite 필요할 때만 시행